In [8]:
# ------------------------------ Produce 4-band matrices from AIII class ------------------------------

import numpy as np
from scipy import linalg as LA
import copy

In [9]:
# Pauli matrices
sigma0 = np.array([[1, 0], [0, 1]], dtype=complex)
sigmax = np.array([[0, 1], [1, 0]], dtype=complex)
sigmay = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigmaz = np.array([[1, 0], [0, -1]], dtype=complex)

tau0 = np.array([[1, 0], [0, 1]], dtype=complex)
taux = np.array([[0, 1], [1, 0]], dtype=complex)
tauy = np.array([[0, -1j], [1j, 0]], dtype=complex)
tauz = np.array([[1, 0], [0, -1]], dtype=complex)




# ------------------------------ The n-band chiral matrix ------------------------------
N_bands = 4

U_CH = np.zeros((N_bands, N_bands), dtype = complex)

for i in range(round(N_bands/2)):
    U_CH[i, i] = 1
    U_CH[round(N_bands/2) + i, round(N_bands/2) + i] = -1
    
U_CH_dag = np.conj(np.transpose(U_CH))

In [20]:
# ---------- Create a random Hermitian matrix ------------
def random_matrix(N_bands):
    
    # Random Hermitian matrix
    A = np.zeros((N_bands, N_bands), dtype = complex)
    
    # Generate a random Hermitian matrix
    for i in range(N_bands):
        A[i, i] = (2*np.random.random()-1)
    
    return A

# ---------- Create a random chirally-symmetric state ----------
def create_random(N_k, n, N_bands, U_CH):

    
    # initialize random matrices Fourier components (from -n to n)
    H_sin = np.zeros((2 * n + 1, N_bands, N_bands), dtype = complex)
    H_cos = np.zeros((2 * n + 1, N_bands, N_bands), dtype = complex)
    
    # run over Fourier components
    for i_Fourier in range(2 * n + 1):
        H_sin[i_Fourier, :, :] = random_matrix(N_bands)
        H_cos[i_Fourier, :, :] = random_matrix(N_bands)            
            
    
    # initialize the state
    H = np.zeros((N_k, N_bands, N_bands), dtype = complex)
    H_normalized = np.zeros((N_k, N_bands, N_bands), dtype = complex)
    
    # run over momentum points in 1d
    for i_x in range(N_k):
        
        # momentum
        k_x = 2 * np.pi * i_x / (N_k)
        
        # add all Fourier components at the corresponding momentum point    
        A = np.zeros((N_bands,N_bands), dtype = complex)
        for i_Fourier in range(n + 1):

            A += H_sin[i_Fourier, :, :] * np.sin(i_Fourier * k_x)
            A += H_cos[i_Fourier, :, :] * np.cos(i_Fourier * k_x) 
         
        # add to the state matrix and symmetrize it
        H[i_x, :, :] += A
        H[i_x, :, :] -= np.matmul(U_CH, np.matmul(A, U_CH_dag))
    
    
    # ---------------- compute eigenvectors  --------------------
    psi = np.zeros((N_k, N_bands, N_bands), dtype = complex)        
    
    D = np.zeros(( psi.shape[1],  psi.shape[1]), dtype = complex)
    for i in range(round(psi.shape[1]/2)):
        D[i, i] = 1
        D[i + round( psi.shape[1]/2), i + round( psi.shape[1]/2)] = -1
         
    for i_x in range(N_k):
            
        eigenValues, eigenVectors = LA.eigh(H[i_x, :, :])
        idx = eigenValues.argsort()[::-1]   
        eigenValues = np.sign(eigenValues[idx])
        psi[i_x, :, :] = eigenVectors[:,idx]     
    
        H_normalized[i_x, :, :] = np.matmul(psi[i_x, :, :], np.matmul(D, np.conj(np.transpose(psi[i_x, :, :]))))    
            
            
    return H_normalized, psi


# ------------ Create a trivial-atomic state ------------
def create_trivial_atomic(N_k, N_bands, H_trivial_atomic):
    
    # initialize
    H = np.zeros((N_k, N_bands, N_bands), dtype = complex)
    psi = np.zeros((N_k, N_bands, N_bands), dtype = complex) 
    
    # decompose
    eigenValues, eigenVectors = LA.eigh(H_trivial_atomic)
    idx = eigenValues.argsort()[::-1]   
    eigenValues = np.sign(eigenValues[idx])
    
    for i_x in range(N_k):
            
        psi[i_x, :, :] = eigenVectors[:,idx] 
        H[i_x, :, :] = H0
          

    return H, psi
                             

In [28]:
# ------------ Distance between psi1 and psi2 (see the corresponding paper) ------------
def distance(psi1, psi2):
    
    N_bands = psi1.shape[0]
    
    d = 0
    for i in range(round(N_bands/2)):
        for j in range(round(N_bands/2)):
            d += (abs(np.matmul(np.conj(np.transpose(psi1[:, i])), psi2[:, j]))**2)/(0.5 * N_bands)

    return (1 - d)

# ------------ Check the chiral symmetry (defined for the eigenstates) ------------
def symmetry_psi(psi, U_CH):
    
    for i_k in range(psi.shape[0]):
        if distance(psi[i_k], np.matmul(U_CH, np.matmul(psi[i_k], U_CH))) > 0.001: 
            return False
            
    return True
    
# ------------ Check the chiral symmetry (defined for the Hamiltonian) ------------
def symmetry_H(H, U_IS):

    if LA.norm(H +  np.matmul(U_IS, np.matmul(H, U_IS))) > 0.001: 
        return False
            
    return True


# ------------ Check continuity -------------               
def is_continuous(psi, delta):

    N_k_x = psi.shape[0]
    
    for k_x in range(N_k_x):

        if distance(psi[k_x - 1, :, :], psi[k_x, :, :]) > delta:
            return False
            
    return True

In [29]:
# ------------ Produce random chiral-symmetric states ------------

# initialize some parameters
N_k = 100
N_bands = 4
N_samples = 1000
delta = 0.1
eta = 0.1
phi_max = 0.1

# Fourier component cutoff
n = 4

# data veriables
data_psi_random  = np.zeros((N_samples, N_k, N_bands, N_bands), dtype = complex)
data_H_random  = np.zeros((N_samples, N_k, N_bands, N_bands), dtype = complex)

for i in range(N_samples):
    
    # while the generated state is discontinuous
    condition = False
    while condition == False:

        H, psi = create_random(N_k, n, N_bands, U_CH)
        condition = is_continuous(psi, delta)
        
    data_psi_random[i, :, :, :] = psi
    data_H_random[i, :, :, :] = H
    
    # check if the state is continuous and chiral-symmetric
    print(i, symmetry_psi(data_psi_random[i, :, :, :], U_CH), is_continuous(data_psi_random[i, :, :, :], delta))  

0 True True
1 True True
2 True True
3 True True
4 True True
5 True True
6 True True
7 True True
8 True True
9 True True
10 True True
11 True True
12 True True
13 True True
14 True True
15 True True
16 True True
17 True True
18 True True
19 True True
20 True True
21 True True
22 True True
23 True True
24 True True
25 True True
26 True True
27 True True
28 True True
29 True True
30 True True
31 True True
32 True True
33 True True
34 True True
35 True True
36 True True
37 True True
38 True True
39 True True
40 True True
41 True True
42 True True
43 True True
44 True True
45 True True
46 True True
47 True True
48 True True
49 True True
50 True True
51 True True
52 True True
53 True True
54 True True
55 True True
56 True True
57 True True
58 True True
59 True True
60 True True
61 True True
62 True True
63 True True
64 True True
65 True True
66 True True
67 True True
68 True True
69 True True
70 True True
71 True True
72 True True
73 True True
74 True True
75 True True
76 True True
77 True T

595 True True
596 True True
597 True True
598 True True
599 True True
600 True True
601 True True
602 True True
603 True True
604 True True
605 True True
606 True True
607 True True
608 True True
609 True True
610 True True
611 True True
612 True True
613 True True
614 True True
615 True True
616 True True
617 True True
618 True True
619 True True
620 True True
621 True True
622 True True
623 True True
624 True True
625 True True
626 True True
627 True True
628 True True
629 True True
630 True True
631 True True
632 True True
633 True True
634 True True
635 True True
636 True True
637 True True
638 True True
639 True True
640 True True
641 True True
642 True True
643 True True
644 True True
645 True True
646 True True
647 True True
648 True True
649 True True
650 True True
651 True True
652 True True
653 True True
654 True True
655 True True
656 True True
657 True True
658 True True
659 True True
660 True True
661 True True
662 True True
663 True True
664 True True
665 True True
666 Tr

In [ ]:
# ---------- save the generated states ----------------
np.save('phi_random.npy', data_psi_random)
np.save('H_random.npy', data_psi_random)

In [ ]:
# ---------- pick one of the states to be the parent 1 ----------
i_parent = 0
np.save('parent_1_psi.npy', data_psi_random[i_parent])
np.save('parent_1_H.npy', data_H_random[i_parent])

In [ ]:
# ---------- create the parent 0 ----------
H = np.load('AIII_matrices.npy')
A = H[0, 0, :, :]
H_parent_0, psi_parent_0  = create_flat(N_k, N_bands, U_CH, A)

In [33]:
# ---------Bonus: compute the winding number ------------
# --------- (just to verify the selection) --------------
def winding_psi(psi): 
    
    w = 0
    
    N_k = psi.shape[0]
    N_bands = psi.shape[1]
    
    # ----------------- diogonal matrix with entries +/- 1 ------------
    D = np.zeros((N_bands, N_bands), dtype = complex)

    for i in range(round(N_bands/2)):
        D[i, i] = 1
        D[i + round(N_bands/2), i + round(N_bands/2)] = -1
    
    Theta = np.zeros(N_k)
    for i_x in range(N_k) : 
        
        # create a matrix from psi 
        H = np.matmul(psi[i_x, :, :], np.matmul(D, np.conj(np.transpose(psi[i_x, :, :]))))
        
        # compute a determinant of the off-diogonal block 
        z = LA.det(H[round(N_bands/2):, :round(N_bands/2)])
        
        # Angle: -pi to pi
        Theta[i_x] = np.angle(z)  
        
            
            
    for i_x in range(N_k) :
        
        # The angle difference: -2 *pi to 2*pi
        delta_Theta = Theta[i_x] - Theta[i_x - 1] 
        
        if delta_Theta > np.pi:
            delta_Theta -= 2 * np.pi
            
        if delta_Theta < -math.pi:
            delta_Theta += 2 * np.pi
        
        # update the winding number
        w += delta_Theta
              
                               
    w = w / (2 * np.pi)
        
    return w



def winding_H(H): 
    
    w = 0
    
    N_k = H.shape[0]
    N_bands = H.shape[1]
    
    Theta = np.zeros(N_k)
    for i_x in range(N_k) : 
        
        # compute a determinant of the off-diogonal block 
        z = LA.det(H[i_x, round(N_bands/2):, :round(N_bands/2)])
        
        # Angle: -pi to pi
        Theta[i_x] = np.angle(z)  
        
            
            
    for i_x in range(N_k) :
        
        # The angle difference: -2 *pi to 2*pi
        delta_Theta = Theta[i_x] - Theta[i_x - 1] 
        
        if delta_Theta > np.pi:
            delta_Theta -= 2 * np.pi
            
        if delta_Theta < -np.pi:
            delta_Theta += 2 * np.pi
        
        # update the winding number
        w += delta_Theta
              
                               
    w = w / (2 * np.pi)
        
    return w

In [34]:
H_0 = np.load('parent_0_H.npy')
print(winding_H(H_0))

H_1 = np.load('parent_1_H.npy')
print(winding_H(H_1))



0.0
-2.0000000000000004
